In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob

marsImage = Image.open("../images/mars.png")
occ_view = pd.read_csv("../data/out/occ_view.csv")
spectralclasses = [pd.read_csv(file) for file in glob("../data/out/class_view/*")]
"""soundings = np.genfromtxt("../data/soundings.txt")
grad = np.genfromtxt("../data/grad.txt")
polyres = np.genfromtxt("../data/polyres.txt")
poly_coeff = np.genfromtxt("../data/poly_coeff.txt")
curv = np.genfromtxt("../data/curv.txt")
labels = np.genfromtxt("../data/labels.txt")"""

index = {0: "flat above atmosphere", 1: "flat below atmosphere", 2: "flat in atmosphere", 3: "high gradient", 
        4: "negative curvature", 5: "periodic", 6: "positive curvature", 7: "small gradient"}
xedni = {v: k for k, v in index.items()}

print("Class index\n")
for key, value in index.items(): print(f"{key}: {value}")

In [ ]:
with h5py.File("../data/level_1p0a/20181001_000633_1p0a_UVIS_E.h5", "r") as f:
    y = np.array(f["Science/YMean"])
    x = np.array(f["Science/X"])[0]
    alt = np.mean(f["Geometry/Point0/TangentAltAreoid"], axis=1)

In [ ]:
plt.plot(x, y.T)
plt.ylim([0,1])
plt.show()

In [ ]:
view = int(input("Press 1 to view a class, 2 to view an occultation "))

# class view
if view == 1:
    spectralclass = int(input("Choose class of soundings to view: "))
    
    plttype = int(input("Press 1 to view geolocations, 2 to view altitude distribution: "))
    dest = input("Type 'show' to show or a file path to save to: ")
    
    if plttype == 1:
        plt.scatter(spectralclasses[spectralclass]["lon"], spectralclasses[spectralclass]["lat"], 
                    c=spectralclasses[spectralclass]["alt"])
        plt.grid()
        plt.title(f"Locations of {index[spectralclass]} soundings")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.xticks(range(-180, 181, 30))
        plt.yticks(range(-90, 91, 30))
        plt.xlim(-180, 180)
        plt.ylim(-90, 90)
        plt.tight_layout()
        cbar = plt.colorbar()
        cbar.set_label("Altitudes / km")
        plt.imshow(marsImage, extent=[-180, 180, 90, -90]);
        
        if dest == "show":
            plt.show()
            
        else:
            plt.savefig(dest)
            plt.close()
        
    elif plttype == 2:
        plt.hist(spectralclasses[spectralclass]["alt"])
        plt.xlabel("Altitude")
        plt.ylabel("Frequency")
        
        if dest == "show":
            plt.show()
            
        else:
            plt.savefig(dest)
            plt.close()
            
# Occultation view
elif view == 2:
    name = input("File name of occultation: ")
    print()
    occ = occ_view[occ_view["file name"] == name]
    print(f"Composition of occultation {name}")
    print(f"Total soundings: {occ['total'][0]}")
    for k, v in index.items(): print(f"{v}: {occ[str(k)][0]}")